In [ ]:
import numpy as np
import pandas as pd
import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller
import matplotlib.pyplot as plt
pd.core.common.is_list_like = pd.api.types.is_list_like
import pandas_datareader as pdr
import datetime
import yfinance as yf
import math
import itertools
from itertools import compress
from scipy.optimize import minimize
from scipy.optimize import Bounds
from datetime import datetime
from matplotlib.ticker import FormatStrFormatter
import matplotlib
from matplotlib import cm
from sklearn.preprocessing import StandardScaler # for standardizing the Data
from sklearn.decomposition import PCA # for PCA calculation
from mpl_toolkits.mplot3d import Axes3D
from statsmodels.stats.weightstats import DescrStatsW
matplotlib.get_configdir()
plt.style.use('science')
yf.pdr_override()

In [ ]:
start = datetime(2015, 1, 1)
end = datetime(2021, 1, 1)

In [ ]:
start = datetime(2015, 1, 1)
end = datetime(2021, 1, 1)

tickers = ['LECBF', 'NUPMF', 'ELKMF', 'OBNNF', 'DGMLF', 'RMLRF', 'DRD', 'RNGTF', 'WDOFF', 'CSFFF', 'HL-PB', 'SVLKF', 'SVM', 'JINFF', 'TORXF', 'DPMLF', 'NGD', 'PMNXF', 'OGDCF', 'SILV', 'OCANF', 'SAND', 'STBMF', 'STBMY', 'KNTNF', 'CENX', 'SA', 'RGRNF', 'FSM', 'HCHDF', 'TGCDF', 'IAG', 'KALU', 'HBM', 'CELTF', 'MAG', 'OR', 'FTMNF', 'CSTM', 'PVG', 'CDE', 'EGO', 'EQX', 'HMY', 'BVN', 'TRQ', 'NG', 'CAGDF', 'AGI', 'HL', 'SSRM', 'AWCMF', 'AWCMY', 'AG', 'SCEXF', 'AA', 'AUY', 'OZMLF', 'EDVMF', 'KZMYY', 'BTG', 'CAHPF', 'NESRF', 'PAAS', 'RGLD', 'LUNMF', 'ALMMF', 'GFI', 'ACH', 'KGC', 'NHYDY', 'NHYKF', 'AU', 'JIAXF', 'KL', 'AUCOY', 'FNLPF', 'IMPUY', 'IMPUF', 'SBSW', 'FQVLF', 'AEM', 'NCMGY', 'NCMGF', 'WPM', 'ANFGF', 'FNV', 'OPYGY', 'AGPPF', 'ANGPY', 'GOLD', 'ZIJMY', 'FCX', 'ZIJMF', 'NEM', 'SCCO','GLD', 'IAU', 'GLDM', 'SGOL', 'BAR','SLV', 'SIVR', 'AGQ', 'ZSL','CPER']
df = pdr.get_data_yahoo(tickers, start, end)['Adj Close']
#df.to_csv('C:/Users/ander/Dropbox/Abe finance/Vol.csv')

In [ ]:
benchmark = ['^GSPC','^DJI','^IXIC','GC=F','^VIX', '^IRX']
bm = pdr.get_data_yahoo(benchmark, start, end)['Adj Close']
bm.to_csv('C:/Users/ander/Dropbox/Abe finance/Benchm.csv')

In [ ]:
pd.options.display.max_rows = 120
#print(df.isna().sum())

In [ ]:
#Read data
cap=pd.read_excel("Cap.xlsx")
close=pd.read_csv("Close.csv", index_col=0)
vol=pd.read_csv("Vol.csv", index_col=0)
bm=pd.read_csv("Benchm.csv", index_col=0)
short_cost=pd.read_excel("Short-fees.xlsx")
short_cost=(1+short_cost.iloc[:,0:37]/100)**(1/365)

In [ ]:
turnover=vol.iloc[:,0:85]*close.iloc[:,0:85]
# find rows that meet your criteria and average
avgturnover = turnover.loc['2019-10-01':'2019-12-31', :].mean(axis=0)

In [ ]:
stonks=[]
for i in range(0,len(avgturnover)):
    if avgturnover[i]>1000000 and cap.iloc[0,i]>1000:
        stonks.append(avgturnover.index[i])       

In [ ]:
ETF=['GLD', 'IAU', 'GLDM', 'SGOL', 'BAR','SLV']
df=close.loc[:,stonks+ETF]
df = df.dropna(how='any', axis=1)
df = (np.log(df))

In [ ]:
def stationarity_test(X, cutoff=0.01):
    # H_0 in adfuller is unit root exists (non-stationary)
    # We must observe significant p-value to convince ourselves that the series is stationary
    pvalue = adfuller(X)[1]
    if pvalue < cutoff:
        print('p-value = ' + str(pvalue) + ' The series ' + X.name +' is likely stationary.')
    else:
        print(X.name +' ' + str(round(pvalue,3)))

In [ ]:
for i in range(0,len(df.columns)):
    if df.index.get_loc(df.iloc[:,i].first_valid_index())==0:
        stationarity_test(df.iloc[:df.index.get_loc('2019-12-31'),i])

In [ ]:
def find_cointegrated_pairs(data):
    n = data.shape[1]
    score_matrix = np.zeros((n, n))
    pvalues= []
    keys = data.keys()
    pairs = []
    for i in range(n):
        for j in range(i+1, n):
            S1 = data.iloc[max(df.index.get_loc(df.iloc[:,i].first_valid_index()),df.index.get_loc(df.iloc[:,j].first_valid_index())):df.index.get_loc('2019-12-31'),i]
            S2 = data.iloc[max(df.index.get_loc(df.iloc[:,i].first_valid_index()),df.index.get_loc(df.iloc[:,j].first_valid_index())):df.index.get_loc('2019-12-31'),j]
            result = coint(S1, S2,trend='ct')   #SKAL RETTES TILBAGE;HVIS DEN SKAL KØRES IGEN!!!
            score = result[0]
            pvalue = result[1]
            score_matrix[i, j] = score
            if pvalue < 0.05:
                pairs.append(keys[i]+" " +keys[j])
                pvalues.append(pvalue)
    return score_matrix, pvalues, pairs

In [ ]:
scores, pvalues, pairs = find_cointegrated_pairs(df.iloc[:1+df.index.get_loc('2019-12-31'),:])
len(pairs)

In [ ]:
#Spread and std. errors are estimated in the periode until 2020-01-20
beta=[]
alpha=[]
spreaddf=pd.DataFrame(index=df.index)
for i in range(0,len(pairs)):
    firstobs=0#max(df.index.get_loc(df.loc[:,pairs[i][0]].first_valid_index()),df.index.get_loc(df.loc[:,pairs[i][1]].first_valid_index()))
    obs=len(df)-firstobs
    if firstobs==0:
        firstobs= None
    else:
        firstobs= max(df.loc[:,pairs[i][0]].first_valid_index(),df.loc[:,pairs[i][1]].first_valid_index())
    

    S1=df.loc[firstobs:,pairs[i].split(' ')[0]].to_numpy()
    S1= sm.add_constant(df.loc[firstobs:,pairs[i].split(' ')[0]]).to_numpy()
    S1_partial=df.loc[firstobs:'2020-01-02',pairs[i].split(' ')[0]].to_numpy()
    S1_partial = sm.add_constant(df.loc[firstobs:'2020-01-02',pairs[i].split(' ')[0]]).to_numpy()
    S2_partial=df.loc[firstobs:'2020-01-02',pairs[i].split(' ')[1]].to_numpy()
    S2_partial=S2_partial.reshape(len(S2_partial),1)
    results = sm.OLS(S2_partial, S1_partial).fit()
    spread = df.loc[:,pairs[i].split(' ')[1]] - np.dot(S1,results.params)
    spreaddf[pairs[i].split(' ')[0] + " "+ pairs[i].split(' ')[1]] = spread 
    beta.append(results.params[1])
    alpha.append(results.params[0])

#Normalising the spreads
zdf=[]
for i in range(0,len(pairs)):
    zdf.append(spreaddf.iloc[:df.index.get_loc('2020-01-02'),i].std())

spreaddf=spreaddf.div(zdf)
oldpairsspread=spreaddf

In [ ]:
pairstable=pd.DataFrame(index=pairs)
pairstable['Alpha']=alpha
pairstable['Beta']=beta
pairstable['p-value']=pvalues
pairstable

In [ ]:
##################### Selection crit ########################
#Sum of  squared diff
sortpairs=pd.DataFrame(((spreaddf.iloc[:df.index.get_loc('2020-01-02'),:].diff()) ** 2).sum())
#0-Crossings
crossing=[((np.array(spreaddf.iloc[:df.index.get_loc('2020-01-02')-1,i])*np.array(spreaddf.iloc[1:df.index.get_loc('2020-01-02'),i]))<0).sum() for i in range(0,len(pairs))]
sortpairs['NZC']=crossing

#Selecting pairs that are not in the worst decile of either category
sortpairs['SSD Rank'] = pd.qcut(sortpairs.iloc[:,0], 4,labels = False) 
sortpairs['NZC Rank'] = pd.qcut(sortpairs['NZC'], 4,labels = False) 

l=sortpairs['NZC Rank']>0
a=sortpairs['SSD Rank']<3
pairs=list(compress(pairs, l&a))

beta_temp=[]
for i in range(0,len(pairs)):
    beta_temp.append(beta[spreaddf.columns.get_loc(pairs[i])])
spreaddf=spreaddf.loc[:,pairs]
beta=beta_temp
temp_df=pd.DataFrame(index=df.index, columns=spreaddf.columns)
temp_df.iloc[:,:]=np.array(beta).reshape(1,len(pairs))
beta_norm=temp_df
spread_norm=spreaddf

In [ ]:
#Lav tabel med, estimater Par, (alpha + beta), Rank i SSD og NZC?
pairstable=pd.concat([pairstable, sortpairs], axis=1)


In [ ]:
pairstable.round({'Alpha': 2, 'Beta': 3,'p-value':3,0:1})

In [ ]:
#Rolling spread
rollstart='2017-01-04'

betarollm=pd.DataFrame(index=df.index, columns=spreaddf.columns)
spreaddfroll=pd.DataFrame(index=df.index)
zdfroll=pd.DataFrame(index=df.index, columns=spreaddf.columns)

for i in range(0,len(pairs)):
    S1=df.loc[:,pairs[i].split(' ')[0]].to_numpy()
    S1= sm.add_constant(df.loc[:,pairs[i].split(' ')[0]]).to_numpy()
    S2=(df.loc[:,pairs[i].split(' ')[1]].to_numpy())
    S2=S2.reshape(len(S2),1)
            
    S1_partial=df.loc[:rollstart,pairs[i].split(' ')[0]].to_numpy()
    S1_partial = sm.add_constant(df.loc[:rollstart,pairs[i].split(' ')[0]]).to_numpy()
    S2_partial=df.loc[:rollstart,pairs[i].split(' ')[1]].to_numpy()
    S2_partial=S2_partial.reshape(len(S2_partial),1)
    results = sm.OLS(S2_partial, S1_partial).fit()
    spread = df.loc[:,pairs[i].split(' ')[1]] - (np.dot(S1,results.params))
    spreaddfroll[pairs[i].split(' ')[0] + " "+ pairs[i].split(' ')[1]] = spread 
    betarollm.iloc[:,i]=(results.params[1])
    zdfroll.iloc[:,i]=spreaddfroll.iloc[:df.index.get_loc('2019-12-31')+1,i].std()
        
    for j in range(df.index.get_loc(rollstart), len(df)):
        results = sm.OLS(S2[0:(j+1)], S1[0:(j+1)]).fit()
        spread = S2[:(j+1)]-(np.dot(S1[:(j+1)],results.params)).reshape(j+1,1)
        spreaddfroll.iloc[j,i] = spread[-1]
        betarollm.iloc[j,i]=(results.params[1])
        zdfroll.iloc[j,i]=spreaddfroll.iloc[:j+1,i].std()      

#Normalising the spreads
spreaddfroll=spreaddfroll/zdfroll

In [ ]:
#Rolling spread
rollstart='2020-01-02'

betaroll=pd.DataFrame(index=df.index, columns=spreaddf.columns)
spreaddfroll=pd.DataFrame(index=df.index)
zdfroll=pd.DataFrame(index=df.index, columns=spreaddf.columns)

for i in range(0,len(pairs)):
    S1=df.loc[:,pairs[i].split(' ')[0]].to_numpy()
    S1= sm.add_constant(df.loc[:,pairs[i].split(' ')[0]]).to_numpy()
    S2=(df.loc[:,pairs[i].split(' ')[1]].to_numpy())
    S2=S2.reshape(len(S2),1)
            
    S1_partial=df.loc[:rollstart,pairs[i].split(' ')[0]].to_numpy()
    S1_partial = sm.add_constant(df.loc[:rollstart,pairs[i].split(' ')[0]]).to_numpy()
    S2_partial=df.loc[:rollstart,pairs[i].split(' ')[1]].to_numpy()
    S2_partial=S2_partial.reshape(len(S2_partial),1)
    results = sm.OLS(S2_partial, S1_partial).fit()
    spread = df.loc[:,pairs[i].split(' ')[1]] - (np.dot(S1,results.params))
    spreaddfroll[pairs[i].split(' ')[0] + " "+ pairs[i].split(' ')[1]] = spread 
    betaroll.iloc[:,i]=(results.params[1])
    zdfroll.iloc[:,i]=spreaddfroll.iloc[:df.index.get_loc('2019-12-31')+1,i].std()
        
    for j in range(df.index.get_loc(rollstart), len(df)):
        results = sm.OLS(S2[0:(j+1)], S1[0:(j+1)]).fit()
        spread = S2[:(j+1)]-(np.dot(S1[:(j+1)],results.params)).reshape(j+1,1)
        spreaddfroll.iloc[j,i] = spread[-1]
        betaroll.iloc[j,i]=(results.params[1])
        zdfroll.iloc[j,i]=spreaddfroll.iloc[:j+1,i].std()      

#Normalising the spreads
spreaddfroll=spreaddfroll/zdfroll

In [ ]:
#Half life exp
spread_norm.iloc[:df.index.get_loc('2019-12-31')+1,:]

z_lag = spread_norm.iloc[:df.index.get_loc('2019-12-31')+1,:].shift()
z_ret=spread_norm.iloc[:df.index.get_loc('2019-12-31')+1,:] - z_lag

halflife=[]

for i in range(0,len(spread_norm.columns)):
    z_lag2 = sm.add_constant(z_lag[z_lag.columns[i]])
    model = sm.OLS(z_ret[z_lag.columns[i]],z_lag2, missing='drop')
    res = model.fit()
    halflife.append(-np.log(2) / res.params[1])
halflife

In [ ]:
#Hurst exponent

In [ ]:
#Boillinger
boilstart='2020-01-02'

betaboil=pd.DataFrame(index=df.index, columns=spreaddf.columns)
spreaddfboil=pd.DataFrame(index=df.index)
zdfboil=pd.DataFrame(index=df.index, columns=spreaddf.columns)

for i in range(0,len(pairs)):
    S1=df.loc[:,pairs[i].split(' ')[0]].to_numpy()
    S1= sm.add_constant(df.loc[:,pairs[i].split(' ')[0]]).to_numpy()
    S2=(df.loc[:,pairs[i].split(' ')[1]].to_numpy())
    S2=S2.reshape(len(S2),1)
            
    S1_partial=df.loc[:boilstart,pairs[i].split(' ')[0]].to_numpy()
    S1_partial = sm.add_constant(df.loc[:boilstart,pairs[i].split(' ')[0]]).to_numpy()
    S2_partial=df.loc[:boilstart,pairs[i].split(' ')[1]].to_numpy()
    S2_partial=S2_partial.reshape(len(S2_partial),1)
    results = sm.OLS(S2_partial, S1_partial).fit()
    spread = df.loc[:,pairs[i].split(' ')[1]] - (np.dot(S1,results.params))
    spreaddfboil[pairs[i].split(' ')[0] + " "+ pairs[i].split(' ')[1]] = spread 
    betaboil.iloc[:,i]=(results.params[1])
        
    for j in range(df.index.get_loc(boilstart), len(df)):
        results = sm.OLS(S2[0:(j+1)], S1[0:(j+1)]).fit()
        spread = S2[:(j+1)]-(np.dot(S1[:(j+1)],results.params)).reshape(j+1,1)
        spreaddfboil.iloc[j,i] = spread[-1]
        betaboil.iloc[j,i]=(results.params[1])
        
#MA_ series
lengthwindow=100
mavg_30 = spreaddfboil.rolling(window=lengthwindow, center=False).mean()  
mavg_30.iloc[:lengthwindow,:]=np.array(mavg_30.iloc[lengthwindow,:].to_list()*lengthwindow).reshape(lengthwindow,len(pairs))
mstd_30 = spreaddfboil.rolling(window=lengthwindow, center=False).std()
mstd_30.iloc[:lengthwindow,:]=np.array(mstd_30.iloc[lengthwindow,:].to_list()*lengthwindow).reshape(lengthwindow,len(pairs))

In [ ]:
names=[]
for i in range(0,len(spreaddf.columns)):
    names.append(spreaddf.columns[i].split()[0])
    names.append(spreaddf.columns[i].split()[1])

In [ ]:
from collections import Counter
Counter(names)

In [ ]:
len(pairs)

In [ ]:
#make plot to illustrate point about Reduction of pairs
dato=[]
dato= pd.to_datetime(df.index, format='%Y-%m-%d')

upper=10
lower=-10

fig, axs = plt.subplots(1, 2)
fig.set_size_inches(14, 5)
axs[0].plot(dato, oldpairsspread.loc[:,:] ,color='grey', linewidth=0.6)
axs[0].set_ylim(lower,upper)
axs[0].set_title('(a)', y=-0.10)
axs[0].axvspan('2020-01-02', '2020-12-31', facecolor='0.2', alpha=0.15)

axs[1].plot(dato, spread_norm.loc[:,:],color='grey',linewidth=0.6)
axs[1].set_ylim(lower,upper)
axs[1].set_title('(b)', y=-0.10)
plt.subplots_adjust(wspace=0.1, hspace=0.2)
axs[1].axvspan('2020-01-02', '2020-12-31', facecolor='0.2', alpha=0.15)

plt.setp(axs[0], ylabel='Standardized spread ($z_t$)')


fig.savefig('XXX.pdf', dpi=400)

In [ ]:
## Implemination of Pairs Trading Strategy

In [ ]:
#Hvilke univers undersøges
#Beta names= beta_norm,betaroll,betaboil
#Spread names= spread_norm,spreaddfroll,spreaddfboil
beta=betaroll.iloc[:df.index.get_loc('2019-12-31')+1,:]
investigated=spreaddfroll.iloc[:df.index.get_loc('2019-12-31')+1,:] #df.index.get_loc('2019-12-31')+1

In [ ]:
def heinopair(data,tresh, stoploss,is_bband,mavg__30,mstd__30,stdgap,stpls):
    tradecost=0.003 # 0.001 + 0.002
    tradelog= pd.DataFrame(0, index=data.index, columns=data.columns)
    pairreturn=pd.DataFrame(math.nan, index=data.index, columns=data.columns)
    tradeinf = dict.fromkeys(data.columns)
    closeslice=pd.DataFrame(0, index=data.index, columns=data.columns)
    closeslice=close.iloc[close.index.get_loc(data.index[0]):close.index.get_loc(data.index[-1])+1,:]
    
    #additional parameters with Bollinger band
    if bool(is_bband) == True:
        #stdgap=2
        #stpls=5
        diffspreadMA =data-mavg__30.iloc[mavg__30.index.get_loc(data.index[0]):,:]
    
    for i in range(0,data.shape[1]):
        S1=data.columns[i].split(' ')[0]
        S2=data.columns[i].split(' ')[1]
        stopped_out=0
        for j in range(0,len(data)):
            status=0
            
            #Bollinger Bands
            if bool(is_bband) == True:
                if data.iloc[j,i]>=mavg__30.iloc[j+mavg__30.index.get_loc(data.index[0]),i]+stpls*mstd__30.iloc[j+mavg__30.index.get_loc(data.index[0]),i] or data.iloc[j,i]<=mavg__30.iloc[j+mavg__30.index.get_loc(data.index[0]),i]-stpls*mstd__30.iloc[j+mavg__30.index.get_loc(data.index[0]),i]:
                    stopped_out=1
                elif stopped_out==1 and np.sign(diffspreadMA.iloc[j,i])!=np.sign(diffspreadMA.iloc[j-1,i]):
                    stopped_out=0
                    
                #Faktisk strategi
                if stopped_out==0 and data.iloc[j,i]>=mavg__30.iloc[j+mavg__30.index.get_loc(data.index[0]),i]+stdgap*mstd__30.iloc[j+mavg__30.index.get_loc(data.index[0]),i] or stopped_out==0 and (j>=1 and tradelog.iloc[j-1,i]==1 and data.iloc[j,i]>mavg__30.iloc[j+mavg__30.index.get_loc(data.index[0]),i]):
                    status=1 #betyder short S2, lang S1
                    tradelog.iloc[j,i]=status
                elif stopped_out==0 and data.iloc[j,i]<=mavg__30.iloc[j+mavg__30.index.get_loc(data.index[0]),i]-stdgap*mstd__30.iloc[j+mavg__30.index.get_loc(data.index[0]),i] or stopped_out==0 and(j>=1 and tradelog.iloc[j-1,i]==-1 and data.iloc[j,i]<mavg__30.iloc[j+mavg__30.index.get_loc(data.index[0]),i]):
                    status=-1 #betyder short S1, lang S2
                    tradelog.iloc[j,i]=status
            
            #Not Bollinger Bands but instead the "standard implementation"
            elif bool(is_bband) == False:
                #Stoploss strat
                if abs(data.iloc[j,i])>=stoploss:
                    stopped_out=1
                elif stopped_out==1 and abs(data.iloc[j,i]+data.iloc[j-1,i])!=abs(data.iloc[j,i])+abs(data.iloc[j-1,i]):
                    stopped_out=0
                #Faktisk strategi
                if stopped_out==0 and data.iloc[j,i]>=tresh or stopped_out==0 and (j>=1 and tradelog.iloc[j-1,i]==1 and data.iloc[j,i]>0):
                    status=1 #betyder short S2, lang S1
                    tradelog.iloc[j,i]=status
                elif stopped_out==0 and data.iloc[j,i]<=-tresh or stopped_out==0 and(j>=1 and tradelog.iloc[j-1,i]==-1 and data.iloc[j,i]<0):
                    status=-1 #betyder short S1, lang S2
                    tradelog.iloc[j,i]=status
            
            
            #Praktisk implementering herfra
            if tradelog.iloc[j-1,i]-tradelog.iloc[j,i]!=0 and status!=0:
                info=[]
                
                tradestart=j
                amt_S2= (math.floor(100000/closeslice.iloc[j,closeslice.columns.get_loc(S2)]))
                amt_S1= ((math.floor(beta.iloc[j,i]*closeslice.iloc[j,closeslice.columns.get_loc(S2)]*amt_S2/closeslice.iloc[j,closeslice.columns.get_loc(S1)])))
                limit=closeslice.iloc[j,closeslice.columns.get_loc(S1)]*amt_S1
                if limit>100000:
                    amt_S1= (math.floor(100000/closeslice.iloc[j,closeslice.columns.get_loc(S1)]))
                    amt_S2= ((math.floor(closeslice.iloc[j,closeslice.columns.get_loc(S1)]*amt_S1/(closeslice.iloc[j,closeslice.columns.get_loc(S2)]*beta.iloc[j,i]))))
                
                xtramny=closeslice.iloc[j,closeslice.columns.get_loc(S2)]*amt_S2+closeslice.iloc[j,closeslice.columns.get_loc(S1)]*amt_S1
                
                startval=amt_S2*closeslice.iloc[j,closeslice.columns.get_loc(S2)]+amt_S1*closeslice.iloc[j,closeslice.columns.get_loc(S1)]
                
                info.append(round(startval,2))
                info.append(round(xtramny,2))
                info.append(round(amt_S1*closeslice.iloc[j,closeslice.columns.get_loc(S1)],2))
                info.append(round(amt_S2*closeslice.iloc[j,closeslice.columns.get_loc(S2)],2))
                info.append(tradestart)
                
            elif tradelog.iloc[j,i]!=0:
                caldays = datetime.fromisoformat(closeslice.index.to_series()[j])-datetime.fromisoformat(closeslice.index.to_series()[tradestart])
                shortcost=(-status-1)/(-2)*amt_S2*closeslice.iloc[tradestart,closeslice.columns.get_loc(S2)]*((short_cost[S2][0])**(caldays.days)-1)+(status-1)/(-2)*amt_S1*closeslice.iloc[tradestart,closeslice.columns.get_loc(S1)]*((short_cost[S1][0])**(caldays.days)-1)
                pairreturn.iloc[j,i]=((tradelog.iloc[j-1,i]*((closeslice.iloc[j,closeslice.columns.get_loc(S1)]-closeslice.iloc[tradestart,closeslice.columns.get_loc(S1)])*amt_S1-(closeslice.iloc[j,closeslice.columns.get_loc(S2)]-closeslice.iloc[tradestart,closeslice.columns.get_loc(S2)])*amt_S2)*(1-tradecost)-tradecost*startval-shortcost)/startval)
                
                if (j==len(data)-1):
                    info.append(j)
                    if not (tradeinf[spreaddf.columns[i]]):
                        tradeinf[spreaddf.columns[i]]=[info]
                    else:
                        tradeinf[spreaddf.columns[i]].append(info)

            elif tradelog.iloc[j-1,i]-tradelog.iloc[j,i]!=0 and status==0:
                caldays = datetime.fromisoformat(closeslice.index.to_series()[j])-datetime.fromisoformat(closeslice.index.to_series()[tradestart])
                shortcost=(-status-1)/(-2)*amt_S2*closeslice.iloc[tradestart,closeslice.columns.get_loc(S2)]*((short_cost[S2][0])**(caldays.days)-1)+(status-1)/(-2)*amt_S1*closeslice.iloc[tradestart,closeslice.columns.get_loc(S1)]*((short_cost[S1][0])**(caldays.days)-1)
                pairreturn.iloc[j,i]=((tradelog.iloc[j-1,i]*((closeslice.iloc[j,closeslice.columns.get_loc(S1)]-closeslice.iloc[tradestart,closeslice.columns.get_loc(S1)])*amt_S1-(closeslice.iloc[j,closeslice.columns.get_loc(S2)]-closeslice.iloc[tradestart,closeslice.columns.get_loc(S2)])*amt_S2)*(1-tradecost)-tradecost*startval-shortcost)/startval)
                
                info.append(j)
                if not (tradeinf[spreaddf.columns[i]]):
                    tradeinf[spreaddf.columns[i]]=[info]
                else:
                    tradeinf[spreaddf.columns[i]].append(info)
                    
    return tradelog, pairreturn, tradeinf


#Tradeinfo holds a list for each trade with (1:total invested (short+long amt),2: money "Excess"
#,3: invested in S1 ,4: invested in S2 ,5:day opened, 6: day closed)

In [ ]:
heinotrade,retdf, tradeinf =heinopair(investigated,2,5,False,mavg_30,mstd_30,1.5,5)

In [ ]:
#________________OPtimizer function______________________ 
def tradegain(tresh, stopl, data,stdgap,stpls):
    
    _,retdf, tradeinf=heinopair(data,tresh, stopl,False,mavg_30,mstd_30,stdgap,stpls)
    
    yo=[]
    k=0
    for i in range(0,len(data.keys())):
        if not tradeinf[data.columns[i]]:
            k=1+k

        else:
            for j in range(0,len(tradeinf[data.columns[i]])):
                yo.append(retdf.iloc[tradeinf[data.columns[i]][j][5],i]*tradeinf[data.columns[i]][j][0])

    return sum(yo)

In [ ]:
mongain=tradegain(2,5,investigated,1.5,5)

In [ ]:
mongain

In [ ]:
max360=np.unravel_index(np.argmax(z360), z360.shape)
#print(x180[max180[0]][max180[1]],y180[max180[0]][max180[1]],z180[max180[0]][max180[1]])
print(x360[max360[0]][max360[1]],y360[max360[0]][max360[1]],z360[max360[0]][max360[1]])

In [ ]:
rr_measaure=pd.DataFrame(rr_measaure)
rr_measaures=rr_measaure.groupby([0]).sum()
rr_measaurem=rr_measaure.groupby([0]).mean()
rr_measauref=pd.DataFrame([rr_measaurem[1],rr_measaures[2]])

In [ ]:
rr_measauref.iloc[0,:]=rr_measauref.iloc[0,:].div(rr_measauref.iloc[0,:].sum())
weighted_stats = DescrStatsW(rr_measauref.iloc[1,:], weights=rr_measauref.iloc[0,:], ddof=0)

In [ ]:
print(weighted_stats.mean/weighted_stats.std)

In [ ]:
#Calculate returns for pairs and the port
rr_measaure=[]
dato=[]
dato= pd.to_datetime(investigated.index, format='%Y-%m-%d')
totalret=[]
avgday=[]
avgsize=[]
pr=0
for i in range(0,len(tradeinf.keys())):
        t=0
        n=0
        length=0
        size=0
        if not tradeinf[investigated.columns[i]]:
            avgsize.append(0)
            avgday.append(0)
            totalret.append(0)
        else:
            for j in range(0,len(tradeinf[spreaddf.columns[i]])):
                n=n+tradeinf[spreaddf.columns[i]][j][0]
                t=t+tradeinf[spreaddf.columns[i]][j][0]*(retdf.iloc[tradeinf[spreaddf.columns[i]][j][5],i]+1)
                length=length+tradeinf[spreaddf.columns[i]][j][5]-tradeinf[spreaddf.columns[i]][j][4]
                size=size+tradeinf[spreaddf.columns[i]][j][0]
                rr_measaure.append([i,tradeinf[spreaddf.columns[i]][j][0],retdf.iloc[tradeinf[spreaddf.columns[i]][j][5],i]*100])
            avgsize.append(size/len(tradeinf[spreaddf.columns[i]]))
            avgday.append(length/len(tradeinf[spreaddf.columns[i]]))
            totalret.append((t/n-1)*len(tradeinf[spreaddf.columns[i]]))
            pr=pr+(t-n)/mongain*(t/n-1)*len(tradeinf[spreaddf.columns[i]])
            
totalret.append(pr)

In [ ]:
#Tabel in sample data
handler=[]
for i in range(0,len(tradeinf.keys())):
    if not tradeinf[spreaddf.columns[i]]:
        handler.append(0)
    else:
        handler.append(len(tradeinf[spreaddf.columns[i]]))
    
oversigtstabel=pd.DataFrame(handler, index=spreaddf.columns,columns=['No. of trades'])
oversigtstabel=oversigtstabel.append(pd.Series(name='Total'))
oversigtstabel.iloc[-1,0]=oversigtstabel.iloc[:-1,0].sum(axis=0)
avgday.append(sum(oversigtstabel.iloc[:-1,0]*avgday/oversigtstabel.iloc[-1,0]))
oversigtstabel['Avg. days position open']=avgday #Alternativ
oversigtstabel['Fraction of period open']=oversigtstabel['No. of trades']*oversigtstabel['Avg. days position open']/len(investigated)
oversigtstabel['Fraction of period open'][-1]=1
avgsize.append(sum(oversigtstabel.iloc[:-1,0]*avgsize/oversigtstabel.iloc[-1,0]))
oversigtstabel['Avg. trade size']=avgsize
oversigtstabel['Return']=np.round(np.multiply(totalret,100),2)
oversigtstabel['Return'][-1]=((oversigtstabel['Avg. trade size'][:-1]/oversigtstabel['Avg. trade size'][:-1].sum())*oversigtstabel['Return'][:-1]).sum()
oversigtstabel['Ann. return']=oversigtstabel['Return'].apply(lambda x: ((1+x/100)**(1/round((dato[-1]-dato[0]).days/365))-1)*100)
oversigtstabel['Return (USD)']=oversigtstabel['Avg. trade size']*oversigtstabel['Return']/100
oversigtstabel['Return (USD)'][-1]=mongain

for i in oversigtstabel.columns:
    oversigtstabel[i]=oversigtstabel[i].apply(lambda x: round(x,2))
    
oversigtstabel
#Avg return pr trade

In [ ]:
# _________________Graph dev section below___________


In [ ]:
from cycler import cycler
plt.rc('axes', prop_cycle=(cycler('color', ['tab:red', '#08519c', 'black']) +
                           cycler('linestyle', ['-','-','-'])))

dato=[]
dato= pd.to_datetime(investigated.index, format='%Y-%m-%d')

def abline(slope, intercept,j,k):
    """Plot a line from slope and intercept"""
    x_vals = np.array(axs[j,k].get_xlim())
    y_vals = intercept + slope * x_vals
    axs[j,k].plot(x_vals, y_vals, color='tab:red')

In [ ]:
Res = sm.OLS(dailylist,list(pd.concat([bm_daily.loc[daily_ret.index,bm_daily.columns[2*j+k]]]*len(pairs))),missing='drop').fit() 

In [ ]:
bm=pd.read_csv("Benchm.csv", index_col=0)
bm.iloc[:,-1]=(((bm.iloc[:,-1]/100+1)**(1/252)-1)*100) #RF rate
bm.iloc[:,-1].fillna(method='ffill')
bm=bm.loc[retdf.index,bm.columns]
#makes a list of daily returns stacking each pair
daily_ret=((retdf+1)/(retdf+1).shift()-1)*100
daily_ret=daily_ret.subtract(bm.iloc[:,-1],axis=0)
bm_daily=(bm/bm.shift()-1)*100
bm_daily=bm_daily.subtract(bm.iloc[:,-1],axis=0)

dailylist=[]
for i in range(0,daily_ret.shape[1]):
    dailylist.append(daily_ret.iloc[:,i].values.tolist())
dailylist = [item for sublist in dailylist for item in sublist]

#heatmap, xedges, yedges = np.histogram2d(good_x,good_y, bins=[120,250])
#extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]


fig, axs = plt.subplots(2, 2)
fig.set_size_inches(15,10)
bogstaver=['(a)','(b)','(c)','(d)']

for j in range(0,2):
    for k in range(0,2):
        for i in range(0,len(pairs)):
#             bad_indices = np.isnan(len(pd.concat([bm_daily.loc[daily_ret.index,bm_daily.columns[2*j+k]]]*len(pairs)))) | np.isnan(dailylist)
#             good_indices = ~bad_indices
#             good_x = pd.concat([bm_daily.loc[daily_ret.index,bm_daily.columns[2*j+k]]]*len(pairs))[good_indices]
#             good_y = list(compress(dailylist,good_indices))

            Res = sm.OLS(dailylist,sm.add_constant(list(pd.concat([bm_daily.loc[daily_ret.index,bm_daily.columns[2*j+k]]]*len(pairs)))),missing='drop').fit(cov_type='HC0') 
            axs[j,k].scatter(bm_daily.loc[daily_ret.index,bm_daily.columns[2*j+k]],daily_ret.iloc[:,i],s=0.5,alpha=0.6, color='dimgray')
            axs[j,k].axvline(x=0,color='black', linewidth=0.6)
            axs[j,k].hlines(0,-5, 5,color='black',linewidth=0.6)
#             slope, intercept = np.polyfit(good_x, good_y, 1)
            abline(Res.params[1],Res.params[0],j,k)
            textstr = '\n'.join((r'$\alpha=\underset{[%.3f]}{%.2f}$' % (Res.bse[0],Res.params[0]),r'$\beta=\underset{[%.3f]}{%.3f}$' % (Res.bse[1],Res.params[1])))
            axs[j,k].text(2.5, 3.5, textstr, fontsize=10, verticalalignment='top',bbox=dict(facecolor='none', edgecolor='black', pad=5, linewidth=0.4),linespacing = 1.5)
            axs[j,k].set_ylim(-4,4)
            axs[j,k].set_xlim(-4,4)
            axs[j,k].set_title(bogstaver[2*j+k], y=-0.12)
plt.subplots_adjust(wspace=0.1, hspace=0.15)
fig.savefig('corr_full.pdf', dpi=200)

In [ ]:
results = sm.OLS(bm_daily.iloc[:,0],bm_daily.iloc[:,2], missing='drop').fit(cov_type='HC0')
results.params

In [ ]:
results = sm.OLS(bm_daily.iloc[:len(daily_ret),0],bm_daily.iloc[:len(daily_ret),1], missing='drop').fit()
results.params

In [ ]:
plt.rcParams["figure.dpi"] = 100
for i in range(0,len(pairs)):
    retdf.iloc[:,i].plot.line(figsize=(10,5),legend=False)

In [ ]:
upper=4
lower=-4

fig, axs = plt.subplots(2, 2)


fig.set_size_inches(10, 5)
axs[0, 0].plot(dato, heinotrade.iloc[:,6],color='black',linewidth=1.2)
axs[0, 0].plot(dato, investigated.iloc[:,6],color='tab:red',linewidth=0.8)
axs[0, 0].plot(dato, retdf.iloc[:,6]*10,color='green',linewidth=0.8)
axs[0, 0].axvspan('2020-01-02', '2020-12-31', facecolor='0.2', alpha=0.1)
axs[0, 0].set_ylim(lower,upper)
#axs[0, 0].title.set_text("(a)",y=1.25)
axs[0, 0].set_title(pairs[6], y=1)
axs[0, 1].plot(dato, heinotrade.iloc[:,-1],color='black',linewidth=1.2)
axs[0, 1].plot(dato, investigated.iloc[:,-1],color='tab:red',linewidth=0.8)
axs[0, 1].plot(dato, retdf.iloc[:,-1]*10,color='green',linewidth=0.8)
axs[0, 1].axvspan('2020-01-02', '2020-12-31', facecolor='0.2', alpha=0.1)
axs[0, 1].set_ylim(lower,upper)
axs[0, 1].set_title(pairs[-1], y=1)
axs[1, 0].plot(dato, heinotrade.iloc[:,-3],color='black',linewidth=1.2)
axs[1, 0].plot(dato, investigated.iloc[:,-3],color='tab:red',linewidth=0.8)
axs[1, 0].plot(dato, retdf.iloc[:,-3]*10,color='green',linewidth=0.8)
axs[1, 0].axvspan('2020-01-02', '2020-12-31', facecolor='0.2', alpha=0.1)
axs[1, 0].set_ylim(lower,upper)
axs[1, 0].set_title(pairs[-3], y=1)
axs[1, 1].plot(dato, heinotrade.iloc[:,-2],color='black',linewidth=1.2)
axs[1, 1].plot(dato, investigated.iloc[:,-2],color='tab:red',linewidth=0.8)
axs[1, 1].plot(dato, retdf.iloc[:,-2]*10,color='green',linewidth=0.8)
axs[1, 1].axvspan('2020-01-02', '2020-12-31', facecolor='0.2', alpha=0.1)
axs[1, 1].set_ylim(lower,upper)
#axs[1, 1].title.set_text("(d)")
axs[1, 1].set_title(pairs[-2], y=1)
plt.subplots_adjust(wspace=0.10, hspace=0.25)

fig.savefig('BTradeoverview.pdf', dpi=400)

In [ ]:
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    print(heinotrade.loc[:,pairs[5]])

In [ ]:
parnr=6
lul=retdf.iloc[df.index.get_loc('2018-07-10'):df.index.get_loc('2018-12-20'),parnr]*10
heinotrade.iloc[df.index.get_loc('2018-07-10'):df.index.get_loc('2018-12-20'),parnr].plot.line(figsize=(10,5),legend=False,color='black',linewidth=1.2)
investigated.iloc[df.index.get_loc('2018-07-10'):df.index.get_loc('2018-12-20'),parnr].plot.line(figsize=(10,5),legend=False,color='tab:red',linewidth=0.8)
lul.plot.line(figsize=(10,5),legend=False,color='green',linewidth=0.8)
plt.savefig('Fredetrade.pdf', dpi=400)

In [ ]:
#Bollinger Bands graph
boilnr=2
mavg_30.iloc[:,boilnr].plot.line()
spreaddfboil.iloc[:,boilnr].plot.line()
lul1=mavg_30.iloc[:,boilnr]+2*mstd_30.iloc[:,boilnr]
lul2=mavg_30.iloc[:,boilnr]-2*mstd_30.iloc[:,boilnr]
lul1.plot.line()
lul2.plot.line(figsize=(10,5), color='black')

In [ ]:
#Boil
upper=2
lower=-2
rightl=-0.5
rightu=0.5

fig, axs = plt.subplots(4, 2)


#fig.set_size_inches(10, 5)
axs[0, 0].plot(dato, heinotrade.iloc[:,6],color='black',linewidth=1.2)
axs[0, 0].plot(dato, investigated.iloc[:,6],color='tab:red',linewidth=0.8)
axs[0, 0].plot(dato, retdf.iloc[:,6]*10,color='green',linewidth=0.8)
axs[0, 0].axvspan('2020-01-02', '2020-12-31', facecolor='0.2', alpha=0.1)
axs[0, 0].set_ylim(lower,upper)
#band
axs[0, 1].plot(dato, mavg_30.iloc[:,6],color='#08519c')
axs[0, 1].plot(dato, spreaddfboil.iloc[:,6],color='tab:red',linewidth=0.8)
axs[0, 1].plot(dato, mavg_30.iloc[:,6]+1.5*mstd_30.iloc[:,6],color='black',linestyle='--',linewidth=0.8)
axs[0, 1].plot(dato, mavg_30.iloc[:,6]-1.5*mstd_30.iloc[:,6],color='black',linestyle='--',linewidth=0.8)
axs[0, 1].axvspan('2020-01-02', '2020-12-31', facecolor='0.2', alpha=0.1)
axs[0, 1].set_ylim(rightl,rightu)
axs[0,1].legend(['Spread 360d MA','Spread', 'Entry bands'], fontsize=8) #

axs[1, 0].plot(dato, heinotrade.iloc[:,-1],color='black',linewidth=1.2)
axs[1, 0].plot(dato, investigated.iloc[:,-1],color='tab:red',linewidth=0.8)
axs[1, 0].plot(dato, retdf.iloc[:,-1]*10,color='green',linewidth=0.8)
axs[1, 0].axvspan('2020-01-02', '2020-12-31', facecolor='0.2', alpha=0.1)
axs[1, 0].set_ylim(lower,upper)
axs[1, 1].plot(dato, mavg_30.iloc[:,-1],color='#08519c')
axs[1, 1].plot(dato, spreaddfboil.iloc[:,-1],color='tab:red',linewidth=0.8)
axs[1, 1].plot(dato, mavg_30.iloc[:,-1]+1.5*mstd_30.iloc[:,-1],color='black',linestyle='--',linewidth=0.8)
axs[1, 1].plot(dato, mavg_30.iloc[:,-1]-1.5*mstd_30.iloc[:,-1],color='black',linestyle='--',linewidth=0.8)
axs[1, 1].axvspan('2020-01-02', '2020-12-31', facecolor='0.2', alpha=0.1)
axs[1, 1].set_ylim(rightl,rightu)
axs[1,1].legend(['Spread 360d MA','Spread', 'Entry bands'], fontsize=8) #

axs[2, 0].plot(dato, heinotrade.iloc[:,-3],color='black',linewidth=1.2)
axs[2, 0].plot(dato, investigated.iloc[:,-3],color='tab:red',linewidth=0.8)
axs[2, 0].plot(dato, retdf.iloc[:,-3]*10,color='green',linewidth=0.8)
axs[2, 0].axvspan('2020-01-02', '2020-12-31', facecolor='0.2', alpha=0.1)
axs[2, 0].set_ylim(lower,upper)
axs[2, 1].plot(dato, mavg_30.iloc[:,-3],color='#08519c')
axs[2, 1].plot(dato, spreaddfboil.iloc[:,-3],color='tab:red',linewidth=0.8)
axs[2, 1].plot(dato, mavg_30.iloc[:,-3]+1.5*mstd_30.iloc[:,-3],color='black',linestyle='--',linewidth=0.8)
axs[2, 1].plot(dato, mavg_30.iloc[:,-3]-1.5*mstd_30.iloc[:,-3],color='black',linestyle='--',linewidth=0.8)
axs[2, 1].axvspan('2020-01-02', '2020-12-31', facecolor='0.2', alpha=0.1)
axs[2, 1].set_ylim(rightl,rightu)
axs[2,1].legend(['Spread 360d MA','Spread', 'Entry bands'], fontsize=8) #


axs[3, 0].plot(dato, heinotrade.iloc[:,-2],color='black',linewidth=1.2)
axs[3, 0].plot(dato, investigated.iloc[:,-2],color='tab:red',linewidth=0.8)
axs[3, 0].plot(dato, retdf.iloc[:,-2]*10,color='green',linewidth=0.8)
axs[3, 0].axvspan('2020-01-02', '2020-12-31', facecolor='0.2', alpha=0.1)
axs[3, 0].set_ylim(lower,upper)
axs[3, 1].plot(dato, mavg_30.iloc[:,-2],color='#08519c')
axs[3, 1].plot(dato, spreaddfboil.iloc[:,-2],color='tab:red',linewidth=0.8)
axs[3, 1].plot(dato, mavg_30.iloc[:,-2]+1.5*mstd_30.iloc[:,-2],color='black',linestyle='--',linewidth=0.8)
axs[3, 1].plot(dato, mavg_30.iloc[:,-2]-1.5*mstd_30.iloc[:,-2],color='black',linestyle='--',linewidth=0.8)
axs[3, 1].axvspan('2020-01-02', '2020-12-31', facecolor='0.2', alpha=0.1)
axs[3, 1].set_ylim(rightl,rightu)
axs[3, 1].legend(['Spread 360d MA','Spread', 'Entry bands'], fontsize=8) #

#axs[1, 1].title.set_text("(d)")
plt.subplots_adjust(wspace=0.10, hspace=0.35)

axs[0,0].text('2020-11-02', 2.5, pairs[6], fontsize=14, verticalalignment='top',bbox=dict(facecolor='none', edgecolor='black', pad=5, linewidth=0),linespacing = 1.5)
axs[1,0].text('2020-11-02', 2.5, pairs[-1], fontsize=14, verticalalignment='top',bbox=dict(facecolor='none', edgecolor='black', pad=5, linewidth=0),linespacing = 1.5)
axs[2,0].text('2020-11-02', 2.5, pairs[-3], fontsize=14, verticalalignment='top',bbox=dict(facecolor='none', edgecolor='black', pad=5, linewidth=0),linespacing = 1.5)
axs[3,0].text('2020-11-02', 2.5, pairs[-2], fontsize=14, verticalalignment='top',bbox=dict(facecolor='none', edgecolor='black', pad=5, linewidth=0),linespacing = 1.5)

fig.savefig('Boil_Tradeoverview.pdf', dpi=400)

In [ ]:
fig, axs = plt.subplots(6, 2)
k=0
for i in range(0,6):
    for j in range(0,2):
        axs[i,j].yaxis.set_major_locator(ticker.MaxNLocator(integer=True))
        axs[i,j].locator_params(axis='y', nbins=4)
        axs[i,j].plot(dato, close.loc[:,pairs[k].split(' ')[0]],linewidth=0.8)
        lul=(close.loc[:,pairs[k].split(' ')[1]]-close.loc[:,pairs[k].split(' ')[1]].mean())/(close.loc[:,pairs[k].split(' ')[1]].std())*close.loc[:,pairs[k].split(' ')[0]].std()+close.loc[:,pairs[k].split(' ')[0]].mean()
        axs[i,j].plot(dato, lul,linewidth=0.8)
        axs[i,j].legend(pairs[k].split(' '),loc=2,fontsize=8)
        k=1+k
        
plt.subplots_adjust(wspace=0.1, hspace=0.2)
fig.savefig('Prices_matchbymeanandstd.pdf', dpi=300)

In [ ]:
from pylab import MaxNLocator
import matplotlib.ticker as ticker

                         
fig, axs = plt.subplots(6, 2)
k=0
for i in range(0,6):
    for j in range(0,2):
        axs[i,j].yaxis.set_major_locator(ticker.MaxNLocator(integer=True))
        axs[i,j].locator_params(axis='y', nbins=4)
        axs[i,j].plot(dato, close.loc[:,pairs[k].split(' ')],linewidth=0.8)
        axs[i,j].legend(pairs[k].split(' '),loc=2,fontsize=8)
        k=1+k
        
plt.subplots_adjust(wspace=0.1, hspace=0.2)



fig.savefig('Allpairsprice.pdf', dpi=300)

In [ ]:
fig, axs = plt.subplots(2, 2)

for i in range(0,2):
    for j in range(0,2):
        axs[i,j].yaxis.set_major_locator(ticker.MaxNLocator(integer=True))

fig.set_size_inches(10, 5)
axs[0, 0].plot(dato, close.loc[:'2019-12-31',['PVG', 'KGC']],linewidth=0.8)
axs[0, 0].set_title('(a)', y=-0.20)
axs[0, 1].plot(dato, close.loc[:'2019-12-31',['HMY', 'AG']],linewidth=0.8)
axs[0, 1].set_title('(b)', y=-0.20)
#axs[0, 1].set_ylim(0,6.5)
axs[1, 0].plot(dato, close.loc[:'2019-12-31',['GFI', 'IAU']],linewidth=0.8)
axs[1, 0].set_title('(c)', y=-0.20)
axs[1, 1].plot(dato, close.loc[:'2019-12-31',['KGC', 'GOLD']],linewidth=0.8)
axs[1, 1].set_title('(d)', y=-0.20)
#axs[1, 1].set_ylim(0.5,4)
plt.subplots_adjust(wspace=0.1, hspace=0.2)
axs[0, 0].legend(['PVG', 'KGC'],loc=2,fontsize=8)
axs[0, 1].legend(['HMY', 'AG'],loc=2,fontsize=8)
axs[1, 0].legend(['GFI', 'IAU'],loc=2,fontsize=8)
axs[1, 1].legend(['KGC', 'GOLD'],loc=2,fontsize=8)



fig.savefig('Fullprices.png', dpi=300)

In [ ]:
#Regne eksempel for at overbevise om ekstreme afkast
tradeinf['BTG NEM']
df.loc[:,['BTG','NEM']]
S1_partial = sm.add_constant(df.loc[firstobs:'2020-01-02','BTG']).to_numpy()
S2_partial=df.loc[firstobs:'2020-01-02','NEM'].to_numpy()
S2_partial=S2_partial.reshape(len(S2_partial),1)
results = sm.OLS(S2_partial, S1_partial).fit()
results.params
#return
((close['BTG'][457]-close['BTG'][266])*beta[12]*100/close['BTG'][266]-(close['NEM'][457]-close['NEM'][266])*100/close['NEM'][266])/(close['BTG'][266]*beta[12]*100/close['BTG'][266]+close['NEM'][266]*100/close['NEM'][266])
#penge tjent på position
((close['BTG'][457]-close['BTG'][266])*beta[12]*100/close['BTG'][266]-(close['NEM'][457]-close['NEM'][266])*100/close['NEM'][266])

In [ ]:
spread1 = df.loc[:'2020-01-02','NEM'] - np.dot(S1_partial,results.params)

In [ ]:
spread1=spread1/spread1.std()
(df['NEM'][266]-(df['BTG'][266]*results.params[1]+results.params[0]))/spread1.std()

In [ ]:
results.params

In [ ]:
(close['BTG'][457]-close['BTG'][266])*beta[12]*100/close['BTG'][266]

In [ ]:
print(close['BTG'][457],close['BTG'][266])
print(close['NEM'][457],close['NEM'][266])

print(close['BTG'][457],close['BTG'][266],beta[12])

In [ ]:
#return
((close['BTG'][457]-close['BTG'][266])*beta[12]*100/close['BTG'][266]-(close['NEM'][457]-close['NEM'][266])*100/close['NEM'][266])

In [ ]:
df.loc[:,['BTG','NEM']].plot.line(figsize=(10,5),legend=False)

In [ ]:
N = 12
plt.rcParams["axes.prop_cycle"] = plt.cycler("color", plt.cm.jet(np.linspace(0,1,N))[:,0:3]*0.75)


plt.plot(dato,betaroll) #, ,color=my_cmap
plt.plot(dato,betarollm, linestyle='--')
plt.axvspan(betaroll.index[1258],betaroll.index[-1], facecolor='0.2', alpha=0.1)
plt.legend(betaroll.columns,loc='center left', bbox_to_anchor=(1.0, 0.5))
plt.ylabel('Hedge ratio')
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(10, 5)
plt.savefig('beta_rolls.pdf', dpi=300)

In [ ]:
#Net Cash position and number of trades
netcash=pd.DataFrame(0, index=spreaddf.index, columns=['Cash position'])
k=0
for i in range(0,len(tradeinf.keys())):
        if not tradeinf[spreaddf.columns[i]]:
            k=1+k

        else:
            for j in range(0,len(tradeinf[spreaddf.columns[i]])):
                netcash.iloc[tradeinf[spreaddf.columns[i]][j][4]:tradeinf[spreaddf.columns[i]][j][5],0]=netcash.iloc[tradeinf[spreaddf.columns[i]][j][4]:tradeinf[spreaddf.columns[i]][j][5],0]+tradeinf[spreaddf.columns[i]][j][1]
    
netcash['Total trades']=heinotrade.abs().sum(axis=1)

#Tid hvor porteføljen ikke er investeret
1-(netcash['Total trades'] == 0).astype(int).sum()/df.index.get_loc('2019-12-31')

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
fig.set_size_inches(10, 5)
ax1.plot(dato, netcash.iloc[:,0]/100000, alpha=0.8)
ax1.plot(dato, netcash.iloc[:,1], color='#08519c', alpha=0.8)
ax2.plot(dato, bm.iloc[:,4], color='black', linewidth=1, linestyle=':')
ax1.axvspan('2020-01-02', '2020-12-31', facecolor='0.2', alpha=0.1) #f'{value:,}'
textstr = '\n'.join((r'Average USD invested:  ' "{0:,.0f}".format(netcash.iloc[:,0].mean()),r'Average number of pair open: {%.2f}' % (netcash.iloc[:,1].mean())))
ax1.text('2018-01-02', 10, textstr, fontsize=10, verticalalignment='top',bbox=dict(facecolor='none', edgecolor='black', pad=5, linewidth=0),linespacing = 1.5)
fig.legend(['USD invested (100k)', '\# pairs open','Out-of-sample','VIX (rh. axis)'], loc='upper right', bbox_to_anchor=(0.69, 0.8))
fig.savefig('Cash_invested.pdf', dpi=300)

In [ ]:
print(y360[11][11])
print(x360[11][11])
print(z360[11][11])

In [ ]:
print(y180[11][11])
print(x180[11][11])
print(z180[11][11])

In [ ]:
z360 = np.array([[tradegain(2,5,investigated,(1+i)/6,(2+i)/6+j/4) for i in range(0,20)] for j in range(0,20)])
x360 = np.array([[(1+i)/6 for i in range(0,20)] for j in range(0,20)])
y360= np.array([[(2+i)/6+j/4 for i in range(0,20)] for j in range(0,20)])

In [ ]:
#plot different stop losses and std error bands for different 
fig = plt.figure()
ax180 = fig.add_subplot(222, projection='3d')
ax180.plot_surface(x180, y180, z180,cmap=cm.Blues, alpha=0.9)
ax180.view_init(elev=0., azim=-90)
ax180.set_ylabel('Stop loss')
ax180.set_xlabel('Standard error band')
#max180=np.unravel_index(np.argmax(z180), z180.shape)
#ax180.plot(x180[max180[0]][max180[1]],y180[max180[0]][max180[1]],z180[max180[0]][max180[1]], markerfacecolor='r', markeredgecolor='r', marker='o', markersize=7)

ax120 = fig.add_subplot(223, projection='3d')
ax120.plot_surface(x120, y120, z120,cmap=cm.Blues, alpha=0.9)
ax120.view_init(elev=0., azim=-90)
ax120.set_ylabel('Stop loss')
ax120.set_xlabel('Standard error band')

ax80 = fig.add_subplot(224, projection='3d')
ax80.plot_surface(x80, y80, z80,cmap=cm.Blues, alpha=0.9)
ax80.view_init(elev=0, azim=-90)
ax80.set_ylabel('Stop loss')
ax80.set_xlabel('Standard error band')

ax360 = fig.add_subplot(221, projection='3d')
ax360.plot_surface(x360, y360, z360,cmap=cm.Blues, alpha=1)
ax360.view_init(elev=0., azim=-90)
ax360.set_ylabel('Stop loss')
ax360.set_xlabel('Standard error band')

ax180.set_title('(b)', y=-0.12)
ax120.set_title('(c)', y=-0.12)
ax80.set_title('(d)', y=-0.12)
ax360.set_title('(a)', y=-0.12)
plt.subplots_adjust(wspace=0.001, hspace=0.1)
plt.rcParams["figure.figsize"] = (10,10)
#plt.savefig('Boil_opt.pdf', dpi=400)
plt.show()

In [ ]:
#plot different stop losses and std error bands for different 
fig = plt.figure()
ax180 = fig.add_subplot(222, projection='3d')
ax180.plot_surface(x180, y180, z180,cmap=cm.Blues, alpha=0.9)
ax180.view_init(elev=0., azim=-90)
ax180.set_xlabel('Standard error band')
#max180=np.unravel_index(np.argmax(z180), z180.shape)
#ax180.plot(x180[max180[0]][max180[1]],y180[max180[0]][max180[1]],z180[max180[0]][max180[1]], markerfacecolor='r', markeredgecolor='r', marker='o', markersize=7)

ax120 = fig.add_subplot(223, projection='3d')
ax120.plot_surface(x120, y120, z120,cmap=cm.Blues, alpha=0.9)
ax120.view_init(elev=0., azim=-90)
ax120.set_xlabel('Standard error band')

ax80 = fig.add_subplot(224, projection='3d')
ax80.plot_surface(x80, y80, z80,cmap=cm.Blues, alpha=0.9)
ax80.view_init(elev=0, azim=-90)
ax80.set_xlabel('Standard error band')

ax360 = fig.add_subplot(221, projection='3d')
ax360.plot_surface(x360, y360, z360,cmap=cm.Blues, alpha=1)
ax360.view_init(elev=0., azim=-90)
ax360.set_xlabel('Standard error band')

ax180.set_title('(b)', y=-0.12)
ax120.set_title('(c)', y=-0.12)
ax80.set_title('(d)', y=-0.12)
ax360.set_title('(a)', y=-0.12)
plt.subplots_adjust(wspace=0.001, hspace=0.1)
plt.rcParams["figure.figsize"] = (10,10)
#plt.savefig('Boil_opt.pdf', dpi=400)
plt.show()

In [ ]:
dailylist=[]
for i in range(0,daily_ret.shape[1]):
    dailylist.append(daily_ret.iloc[:,i].values.tolist())

dailylist = [item for sublist in dailylist for item in sublist]

from itertools import compress
bad_indices = np.isnan(pd.concat([bm.iloc[:len(daily_ret),0]]*len(pairs))) | np.isnan(dailylist)
good_indices = ~bad_indices
good_x = pd.concat([bm.iloc[:len(daily_ret),0]]*len(pairs))[good_indices]
good_y = list(compress(dailylist,good_indices.values.tolist()))

heatmap, xedges, yedges = np.histogram2d(good_x,good_y, bins=[120,250])
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]


plt.figure(figsize=(5, 5))
plt.imshow(heatmap.T,cmap='binary',aspect='auto',extent=extent,origin='lower',vmin=0, vmax=50)
plt.axvline(x=0,color='black')
plt.hlines(0,xedges[0], xedges[-1],color='black')
slope, intercept = np.polyfit(good_x, good_y, 1)
plt.plot(good_x, good_x*slope + intercept, 'r', linewidth=0.1)
plt.ylim(-4,4)
plt.xlim(-4,4)